# Hastings Direct Takehome

Background:
Insurance companies make pricing decisions based on historical claims experience. The more recent the claims experience, the more predictive it may be of future losses. In the case of many large claims however, the exact cost is not known at the time of the accident. In fact, some cases take years to develop and settle. Companies sometimes learn that a claim is large several years after the accident took place.
Your Underwriting Director believes it is possible to predict the ultimate value of individual claims well in advance by using FNOL (First Notification Of Loss) characteristics. This is the information recorded when the claim is first notified. If so, it would allow the company to know about future costs earlier and this information could be used to make better pricing decisions.
You are given a historical dataset of a particular type of claim - head-on collisions - and are also told their individual current estimated values (labelled Incurred). (Given these claims are now a few years old, you can assume the incurred values are equal to the cost at which the claims will finally settle). 

Task breakdown:
1) Using this data, build a model to predict the ultimate individual claim amounts
"2) Prepare a 15 minute presentation summarising your model. Your presentation should either be in notebook format or a more traditional slide deck.  If you opt for the slide deck approach, please make sure that you provide supporting code. 
Your presentation should cover the following aspects:
- Issues identified with the data and how these were addressed
- Data cleansing
- Model specification and justification for selecting this model specification
- Assessment of your model's accuracy and model diagnostics
- Suggestions of how your model could be improved
- Practical challenges for implementing your model"

Note: columns beginning with TP_* show the number of third parties involved in an accident (under a given category)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport


/var/folders/g7/_7cggn5x0b3bf_rty1bqjb3c0000gn/T/ipykernel_11659/2179082641.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [5]:
data = pd.read_csv('data/task_data.csv')

In [17]:
data.head(3).T

,0,1,2
Claim Number,1,2,3
date_of_loss,2003-04-15,2003-04-20,2003-04-24
Notifier,PH,CNF,CNF
Loss_code,LD003,LD003,LD003
Loss_description,Head on collision,Head on collision,Head on collision
Notification_period,22,1,5
Inception_to_loss,13,9,17
Location_of_incident,Main Road,Main Road,Main Road
Weather_conditions,NORMAL,WET,WET
Vehicle_mobile,Y,Y,Y


In [8]:
print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)}")

Dataset shape: (7691, 47)
Columns: ['Claim Number', 'date_of_loss', 'Notifier', 'Loss_code', 'Loss_description', 'Notification_period', 'Inception_to_loss', 'Location_of_incident', 'Weather_conditions', 'Vehicle_mobile', 'Time_hour', 'Main_driver', 'PH_considered_TP_at_fault', 'Vechile_registration_present', 'Incident_details_present', 'Injury_details_present', 'TP_type_insd_pass_back', 'TP_type_insd_pass_front', 'TP_type_driver', 'TP_type_pass_back', 'TP_type_pass_front', 'TP_type_bike', 'TP_type_cyclist', 'TP_type_pass_multi', 'TP_type_pedestrian', 'TP_type_other', 'TP_type_nk', 'TP_injury_whiplash', 'TP_injury_traumatic', 'TP_injury_fatality', 'TP_injury_unclear', 'TP_injury_nk', 'TP_region_eastang', 'TP_region_eastmid', 'TP_region_london', 'TP_region_north', 'TP_region_northw', 'TP_region_outerldn', 'TP_region_scotland', 'TP_region_southe', 'TP_region_southw', 'TP_region_wales', 'TP_region_westmid', 'TP_region_yorkshire', 'Incurred', 'Capped Incurred', 'Unnamed: 46']


Dont have loads of data but some good columns

In [16]:
print("Generating pandas profiling report...")
profile = ProfileReport(
    data, 
    title="Hastings Direct Claims Data - Pandas Profiling Report",
    explorative=True,
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": True},
        "phi_k": {"calculate": True},
        "cramers": {"calculate": True}
    },
    missing_diagrams={
        "matrix": True,
        "bar": True,
        "heatmap": True,
        "dendrogram": True
    },
    duplicates={
        "head": 10
    }
)

Generating pandas profiling report...


In [11]:
output_file = "hastings_data_profile.html"
print(f"Saving report to {output_file}...")
profile.to_file(output_file)
print(f"Report saved as: {output_file}")

Saving report to hastings_data_profile.html...


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Report saved as: hastings_data_profile.html


In [18]:
data.head(3).T

,0,1,2
Claim Number,1,2,3
date_of_loss,2003-04-15,2003-04-20,2003-04-24
Notifier,PH,CNF,CNF
Loss_code,LD003,LD003,LD003
Loss_description,Head on collision,Head on collision,Head on collision
Notification_period,22,1,5
Inception_to_loss,13,9,17
Location_of_incident,Main Road,Main Road,Main Road
Weather_conditions,NORMAL,WET,WET
Vehicle_mobile,Y,Y,Y


In [ ]:
data['date_of_loss'] = pd.to_datetime(data['date_of_loss'])


In [ ]:
category_cols = [
    'Notifier'
    ,'Location_of_incident'
    ,'Weather_conditions'
    ,'Vehicle_mobile'
]

Qs
What does inception to loss mean?

In [ ]:
drop_cols = [
    'Loss_code' # all the same
    ,'Loss_description' # all the same
]

In [19]:
data.dtypes

Claim Number                      int64
date_of_loss                     object
Notifier                         object
Loss_code                        object
Loss_description                 object
Notification_period               int64
Inception_to_loss                 int64
Location_of_incident             object
Weather_conditions               object
Vehicle_mobile                   object
Time_hour                         int64
Main_driver                      object
PH_considered_TP_at_fault        object
Vechile_registration_present     object
Incident_details_present         object
Injury_details_present           object
TP_type_insd_pass_back           object
TP_type_insd_pass_front          object
TP_type_driver                   object
TP_type_pass_back                object
TP_type_pass_front               object
TP_type_bike                     object
TP_type_cyclist                  object
TP_type_pass_multi               object
TP_type_pedestrian               object


TO DO:
Weather_conditions needs missing filling - us N/K value
Time_hour has suspicious amount at 0 - probably missing value so treat that as categorical